In [1]:
import numpy as np

from tvm import relay
from tvm.relay import testing
import tvm
from tvm import te
from tvm.contrib import graph_executor
import tvm.testing

In [2]:
batch_size = 1
num_class = 1000
image_shape = (3, 224, 224)
data_shape = (batch_size,) + image_shape
out_shape = (batch_size, num_class)

mod, params = relay.testing.vgg.get_workload(
    num_layers=11, batch_size=batch_size, image_shape=image_shape
)

# set show_meta_data=True if you want to show meta data
print(mod.astext(show_meta_data=False))

#[version = "0.0.5"]
def @main(%data: Tensor[(1, 3, 224, 224), float32] /* ty=Tensor[(1, 3, 224, 224), float32] */, %conv1_1_weight: Tensor[(64, 3, 3, 3), float32] /* ty=Tensor[(64, 3, 3, 3), float32] */, %conv1_1_bias: Tensor[(64), float32] /* ty=Tensor[(64), float32] */, %conv2_1_weight: Tensor[(128, 64, 3, 3), float32] /* ty=Tensor[(128, 64, 3, 3), float32] */, %conv2_1_bias: Tensor[(128), float32] /* ty=Tensor[(128), float32] */, %conv3_1_weight: Tensor[(256, 128, 3, 3), float32] /* ty=Tensor[(256, 128, 3, 3), float32] */, %conv3_1_bias: Tensor[(256), float32] /* ty=Tensor[(256), float32] */, %conv3_2_weight: Tensor[(256, 256, 3, 3), float32] /* ty=Tensor[(256, 256, 3, 3), float32] */, %conv3_2_bias: Tensor[(256), float32] /* ty=Tensor[(256), float32] */, %conv4_1_weight: Tensor[(512, 256, 3, 3), float32] /* ty=Tensor[(512, 256, 3, 3), float32] */, %conv4_1_bias: Tensor[(512), float32] /* ty=Tensor[(512), float32] */, %conv4_2_weight: Tensor[(512, 512, 3, 3), float32] /* ty=Tensor[

In [3]:
opt_level = 3
target = 'llvm'
with tvm.transform.PassContext(opt_level=opt_level):
    lib = relay.build(mod, target, params=params)

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.
[14:25:07] /home/wendell/Desktop/tvm/src/runtime/threading_backend.cc:320: Warning: more than two frequencies detected!


In [4]:
# create random input
dev = tvm.cpu()
data = np.random.uniform(-1, 1, size=data_shape).astype("float32")
# create module
module = graph_executor.GraphModule(lib["default"](dev))
# set input and parameters
module.set_input("data", data)
# run
module.run()
# get output
out = module.get_output(0, tvm.nd.empty(out_shape)).numpy()

# Print first 10 elements of output
print(out.flatten()[0:10])

[0.00099007 0.00100382 0.00100122 0.0010073  0.00100486 0.00098953
 0.00098409 0.00099507 0.00104262 0.00100149]


In [5]:
from tvm.contrib import utils

temp = utils.tempdir()
path_lib = temp.relpath("deploy_lib.tar")
lib.export_library(path_lib)
print(temp.listdir())

['deploy_lib.tar']


In [6]:
# load the module back.
loaded_lib = tvm.runtime.load_module(path_lib)
input_data = tvm.nd.array(data)

module = graph_executor.GraphModule(loaded_lib["default"](dev))
module.run(data=input_data)
out_deploy = module.get_output(0).numpy()

# Print first 10 elements of output
print(out_deploy.flatten()[0:10])

# check whether the output from deployed module is consistent with original one
tvm.testing.assert_allclose(out_deploy, out, atol=1e-5)

[0.00099007 0.00100382 0.00100122 0.0010073  0.00100486 0.00098953
 0.00098409 0.00099507 0.00104262 0.00100149]
